#### INF8111 - Data Mining

## TP1 FALL 2020 - Duplicate Bug Report Detection

##### Due date: 23/05

##### Team Members:

    - Yuhan Li (1947497) 1
    - Yanis Toubal (1960266) 2
    
    
##### Deliverables:

You must submit two separate files to Moodle:
1. Notebook
2. [Json](https://en.wikipedia.org/wiki/JSON) containing scraped web-page content (bug_reports.json)




## 1 - Overview


Due to complexities of software systems, software bugs are prevalent. Companies, especially large ones, usually use a Bug Tracking System (BTS), also called Issue Tracking System, to manage and track records of bugs. Besides developers and testers, many projects, mainly open source projects, allow users to report new bugs in their BTS.
To do that, users have to fill a form with multiple fields. An important subset of
these fields provides categorical data and only accepts values that range from a fixed list of
options (e.g., component, version and product of the system). Another two important fields
are the summary and the description. The users are free to write anything in both fields
and the only constraint is that the summary has a maximum number of characters. The
submission of a form creates a page, called bug report or issue report, which contains all
the information about a bug.

Due to the lack of communication and syncronization, users may not be aware that a specific bug was already submitted and report it again. Identifying duplicate bug reports is important task in the BTSs and it is the subject of this TP. Basically, our goal is to develop a system that will compare a new bug report with the submitted ones and rank them based on textual similarity. This ranked list will be used by a triager to manually identify whether a report is duplciate or not.

# 2 Setup

Please run the code below to install the packages needed for this assignment.

In [1]:
                                                                                                                                                                                                                # If you want, you can use anaconda and install after nltk library
# pip install --user numpy
# pip install --user sklearn
# pip install --user scipy
# pip install --user nltk
# pip install --user tqdm
# pip install --user lxml


#python
import nltk
nltk.download("punkt")
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\yanis\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yanis\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# 3 - Data

Please download the zip file in the following url: https://drive.google.com/file/d/14BrFaOiVDIcpjwpm5iAkAy0v_moZyGiP/view?usp=sharing

In this zip file, there are: 

1. test.json: This file contains duplicate bug reports that will be used to evaluate our system.
2. bug_reports: It is a folder that contains the bug report html source. Each html file name follows the pattern **bug_report_id.html**.


Figure below depicts an bug report page example:


In [2]:
from IPython import display
display.Image("https://i.ibb.co/X7mHZbY/bug-report.png")

- A : bug report id
- B : creation date
- C : summary
- D : product
- E : component
- F : the report id which the bug report is duplicate
- G : description

The following script loads the test dataset and define some global variables:

In [3]:
import os

# define the folder path that contain the data
# FOLDER_PATH = "Define folder path that contain threads folder and test.json"
FOLDER_PATH = "dataset/"
PAGE_FOLDER = os.path.join(FOLDER_PATH, 'bug_reports')


# Load the evaluation dataset
import json


test = json.load(open(os.path.join(FOLDER_PATH, "test.json")))

# 4 - Web scraping

Web scraping consists in extracting relevant data from pages and prepare it for computational analysis.


## 4.1 - Question 1 (2 points)

Implement extract_data_from_page function. This function extracts the following information from the html: bug report id, creation date, title, product, component, the report id which it is duplicate and description.

*extract_data_from_page* function returns a dictionary with the following structure:


 ```python
{"report_id": int, 
  "dup_id": int or None (the report id which it is duplicate), 
  "component": string, 
  "product": string, 
  "summary": str, 
  "description": string, 
  "creation_date": int}
 ```
For instance, when extract_data_from_page receives "bug_report/7526.html", it returns:
 
```python
{
"report_id": 7526,
"dup_id": 7799,
"product": "core graveyard",
"component":  tracking,
"summary": "Apprunner crashes on exit",
"description": "Apprunner crashes on exit, without fail. The browser window closes, but the
console window hangs around. I end up having to kill winoldap with the \\"Close
Program\\" dialog (Ctrl-Alt-Del).",
"creation_date": 928381771
}
```

**Creation date have to be represented as timestamp. If bug report is not duplicate, dup_id have to be None.**

*HINT: lxml parse is faster than html.parser*

In [4]:
from bs4 import BeautifulSoup

def extract_data_from_page(pagepath):
    """
    Scrap bug report content from bug report html.
    
    :param pagepath: the path of html file.
    :return: 
        {
        "report_id": int,
        "dup_id": int or None (the report id which it is duplicate), 
        "component": string, 
        "product": string, 
        "summary": str, 
        "description": string, 
        "creation_date": int
        }
    """
    #Open and Read html page
    bug_dict = {}
    file_to_read = open(pagepath, "rb")
    page_content = file_to_read.read()
    soup = BeautifulSoup(page_content, 'lxml')
    
    #Report id
    report_id_element = list(soup.find('span', {'id': 'field-value-bug_id'}).children)[1]
    report_id = report_id_element.getText()[4:]
    bug_dict["report_id"] = int(report_id)
    
    #Duplication id
    dup_id_wrapper = soup.find('span', {'id': 'field-value-status-view'})
    dup_id_element = dup_id_wrapper.find('a')
    dup_id = None if dup_id_element is None else dup_id_element.getText()[4:]
    bug_dict["dup_id"] = dup_id
    
    #Product
    product_element = list(soup.find('span', {'id': 'product-name'}).children)[0]
    bug_dict["product"] = product_element.rstrip()
    
    #Component
    component_element = list(soup.find('span', {'id': 'component-name'}).children)[0]
    bug_dict["component"] = component_element.rstrip()
    
    #Summary
    summary = list(soup.find('h1', {'id': 'field-value-short_desc'}).children)[0]
    bug_dict["summary"] = str(summary)
    
    #Description
    description_element = soup.find('pre', {'id': 'ct-0'}).getText()
    bug_dict["description"] = description_element
    
    #Creation date
    creation_date_element = soup.find('span', {'class': 'rel-time'})
    creation_date = creation_date_element['data-time']
    bug_dict["creation_date"] = creation_date
    
    return bug_dict


## 4.3 - Extract text from HTML


In [5]:
import os
from multiprocessing import Pool, TimeoutError
from time import time
import json
import tqdm

# Index each thread by its id
index_path = os.path.join(PAGE_FOLDER, 'bug_reports.json')

if os.path.isfile(index_path):
    # Load threads that webpage content were already extracted.
    report_index = json.load(open(index_path))
else:
    # Extract webpage content

    # This can be slow (around 10 minutes). Test your code with a small sample. lxml parse is faster than html.parser
    files = [os.path.join(PAGE_FOLDER, filename) for filename in os.listdir(PAGE_FOLDER)]
    reports = [extract_data_from_page(f) for f in tqdm.tqdm(files)]
    report_index = dict(((report['report_id'], report) for report in reports ))

    # Save preprocessed bug reports
    json.dump(report_index, open(index_path,'w'))
    

# 5 - Data Preprocessing

Preprocessing is a crucial task in data mining. This task clean and transform the raw data in a format that can better suit data analysis and machine learning techniques. In natural language processing (NLP), *tokenization* and *stemming* are two well known preprocessing steps. Besides these two steps, we will implement an additional step that is designed exclusively for the twitter domain.

## 5.1 - Tokenization

In this preprocessing step, a *tokenizer* is responsible for breaking a text in a sequence of tokens (words, symbols, and punctuations). For instance, the sentence *"It's the student's notebook."* can be split into the following list of tokens: ['It', "'s", 'the', 'student', "'s", 'notebook', '.'].


### 5.1.1 - Question 2 (0.5 point) 

Implement the following functions: 
- **tokenize_space** tokenizes the tokens that are separated by whitespace (space, tab, newline). This is a naive tokenizer.
- **tokenize_nltk** uses the default method of the nltk package (word_tokenize) to tokenize the text. (https://www.nltk.org/api/nltk.html)
- **tokenize_space_punk** replaces the punctuation to space and then tokenizes the tokens that are separated by whitespace (space, tab, newline).




**All tokenizers have to lowercase the tokens.**

## 5.2 - Stop words removal

### 5.2.1 - Question 3 (0.5 points)

There are a set of tokens, call stop words, that are not signficant to the similarity comparison since they appear in most of bug report pages. Thus, removing them decreases the vector dimensionality and turns the similarity calculation computationally cheaper. Describe the tokens that can be removed without affecting the similarity comparison? Moreover, implement the function filter_tokens that removes these words from a list of tokens.

*You can use a predefined set of words.*

**We used Pythons' NLTK library, which provides a set of stopwords. In our case, the words filtered out were: "of", "it", and "the".**

In [6]:
from nltk.tokenize import word_tokenize
import string

def tokenize_space(text):
    """
    Tokenize the tokens that are separated by whitespace (space, tab, newline). 
    We consider that any tokenization was applied in the text when we use this tokenizer.
    
    For example: "hello\tworld of\nNLP" is split in ['hello', 'world', 'of', 'NLP']
    """
    # return a list of tokens
    return [word.lower() for word in text.split()] 
        
def tokenize_nltk(text):
    """
    This tokenizer uses the default function of nltk package (word_tokenize) to tokenize the text. [https://www.nltk.org/api/nltk.tokenize.html]
    """
    # return a list of tokens
    return word_tokenize(text.lower())


def tokenize_space_punk(text):
    """
    This tokenizer replaces punctuation to spaces and then tokenizes the tokens that are separated by whitespace (space, tab, newline).
    """
    text_cleaned = "".join(ch.lower() if ch not in string.punctuation else ' ' for ch in text)
    return tokenize_space(text_cleaned)

#tests
print(tokenize_space("hello\tworld of\nNLP"))
print(tokenize_space("It's the student's notebook."))
print("\n")

print(tokenize_nltk("hello\tworld of\nNLP"))
print(tokenize_nltk("It's the student's notebook."))
print("\n")

print(tokenize_space_punk("hello\tworld of\nNLP"))
print(tokenize_space_punk("It's the student's notebook."))
        

['hello', 'world', 'of', 'nlp']
["it's", 'the', "student's", 'notebook.']


['hello', 'world', 'of', 'nlp']
['it', "'s", 'the', 'student', "'s", 'notebook', '.']


['hello', 'world', 'of', 'nlp']
['it', 's', 'the', 'student', 's', 'notebook']


In [7]:
from nltk.corpus import stopwords

def filter_tokens(tokens):
    stopwords_en = set(stopwords.words('english')) 
    return [token for token in tokens if not token in stopwords_en]

#test
print(filter_tokens(tokenize_space("hello\tworld of\nNLP")))
print(filter_tokens(tokenize_space("It's the student's notebook.")))
print("\n")

print(filter_tokens(tokenize_nltk("hello\tworld of\nNLP")))
print(filter_tokens(tokenize_nltk("It's the student's notebook.")))
print("\n")

print(filter_tokens(tokenize_space_punk("hello\tworld of\nNLP")))
print(filter_tokens(tokenize_space_punk("It's the student's notebook.")))


['hello', 'world', 'nlp']
["student's", 'notebook.']


['hello', 'world', 'nlp']
["'s", 'student', "'s", 'notebook', '.']


['hello', 'world', 'nlp']
['student', 'notebook']


## 5.3 - Stemming

The process to convert tokens with the same stem (word reduction that keeps word prefixes) to a standard form is called *stemming*. For instance, the word "fishing", "fished" , "fish" and "fisher" are reduced to the stem "fish".


In [8]:
from nltk.stem.snowball import SnowballStemmer

stemmer = SnowballStemmer('english')


word1 = ['I', 'tried', 'different', 'fishes']

print([ stemmer.stem(w) for w in word1])

word2 = ['I', 'will', 'tries', 'only', 'one', 'fishing']
print([ stemmer.stem(w) for w in word2])

['i', 'tri', 'differ', 'fish']
['i', 'will', 'tri', 'onli', 'one', 'fish']


### 5.3.1 - Question 4 (0.5 point) 

Explain how stemming can benift our system?

Stemming is a method that reduces the size of our dictionnary since it groups mutiple word forms into one common word. This reduces the complexity of our operation in question, and hence makes our system faster. Additionally, if stemming is done properly, it can lead to more accurate results.

# 6 - Data Representation


# 6.1 - Bag of Words


Many algorithms only accept inputs that have the same size. However, there are some data types whose sizes are not fixed, for instance, a text can have an unlimited number of words. Imagine that we retrieve two sentences: ”Board games are much better than video games” and ”Monopoly is an awesome game!”. These sentences are respectively named as Sentence 1 and 2. Table below depicts how we could represent both sentences using a fixed representation.

|            | an | are | ! | monopoly | awesome | better | games | than | video | much | board | is | game |
|------------|----|-----|---|----------|---------|--------|-------|------|-------|------|-------|----|------|
| Sentence 1 | 0  | 1   | 0 | 0        | 0       | 1      | 2     | 1    | 1     | 1    | 1     | 0  | 0    |
| Sentence 2 | 1  | 0   | 0 | 1        | 1       | 0      | 0     | 0    | 0     | 0    | 0     | 1  | 1    |

Each column of this table 2.1 represents one of 13 vocabulary words, whereas the rows contains the word
frequencies in each sentence. For instance, the cell in row 1 and column 7 has the value 2
because the word games occurs twice in Sentence 1. Since the rows have always 13 values, we
could use those vectors to represent the Sentences 1 and 2. The table above illustrates a technique called bag-of-words. Bag-of-words represents a document as a vector whose dimensions are equal to the number of times that vocabulary words appeared in the document. Thus, each token will be related to a dimension, i.e., an integer.


The matrix above is called **document-term matrix**. Each cell in this matrix represents the number of times
a particular word (defined by the column) occurs in a particular document (defined by
the row). 




### 6.1.1 - Sparse matrix

In real applications, the document-term matrix is usually sparse, i.e., most of the vector values are 0. To reduce memory usage and speed up matrix operations, these vectors are stored using data structures in which only non-zeros entries are kept. You can find more information on https://en.wikipedia.org/wiki/Sparse_matrix.

In this TP, we employ the compressed sparse row (CSR) to represent the document-term matrix. You can find below two  ways to create a CSR matrix using Scipy (https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csc_matrix.html).



In [9]:
############################################
# Correct ways to create a csr_matrix
############################################

"""
Example:	hello	world	!	data	mining
document_1	1	2	0	0	0
document_2	0	4	0	5	6
document_3	1	1	3	0	1
"""

from scipy.sparse import csr_matrix
import numpy as np

# One option

data = np.asarray([1.0,2.0,4.0,5.0,6.0,1.0,1.0,3.0,1.0]) # Non-zeroes values
row = np.asarray([0,0,1,1,1,2,2,2,2]) # row of each element
col = np.asarray([0,1,1,3,4,0,1,2,4]) # column of each element
M = csr_matrix((data, (row, col)), shape=(3, 5))
print(M.toarray())


# More efficient option

"""
A figure exemplifies data, indptr, indices in the following link: 
https://stackoverflow.com/questions/52299420/scipy-csr-matrix-understand-indptr
"""
data = np.asarray([1.0,2.0,4.0,5.0,6.0,1.0,1.0,3.0,1.0]) # Non-zeroes values
indptr = np.array([0, 2, 5, 9]) # (indptr[i-1],indptr[i]) is the column indices in indptr (i.e., indices[indptr[i-1]:indptr[i]]) of the i-th row 
indices = np.array([0,1,1,3,4,0,1,2,4]) # columns indices

M = csr_matrix((data, indices, indptr), shape=(3, 5))
print(M.toarray())

[[1. 2. 0. 0. 0.]
 [0. 4. 0. 5. 6.]
 [1. 1. 3. 0. 1.]]
[[1. 2. 0. 0. 0.]
 [0. 4. 0. 5. 6.]
 [1. 1. 3. 0. 1.]]


You can find below two non-optimized ways to create a csr_matrix. **They can cause memory and performance issues.**

In [10]:
############################################
# Wrong ways to create a csr_matrix
############################################

"""
Example:	hello	world	!	data	mining
document_1	1	2	0	0	0
document_2	0	4	0	5	6
document_3	1	1	3	0	1
"""
from scipy.sparse import csr_matrix
import numpy as np

"""
Append 1 to data and the word column index to indices for each time that a word appears in a document
"""
data = np.asarray([1.,1.,1.,   1.,1.,1.,1., 1.,1.,1.,1.,1., 1.,1.,1.,1.,1.,1.,  1.,1.,1.,1.,1.,1.])
indptr = np.array([0, 3, 18, 24])
indices = np.array([0,1,1,  1,1,1,1, 3,3,3,3,3, 4,4,4,4,4,4,   0,1,2,2,2,4]) # columns indices
M = csr_matrix((data, indices, indptr), shape=(3, 5))# Even though it prints the correct matrix, it might be slow and consume a lot of memory 
print(M.toarray())############################################

"""
Use numpy or lists to create csr_matrix. This uses the same amount of memory of a dense matrix
"""
l = np.asarray([[1.,2.,0.,0.,0.],[0.,4.,0.,5.,6.],[1.,1.,3.,0.,1.]])
M = csr_matrix(l)
print(M.toarray())

[[1. 2. 0. 0. 0.]
 [0. 4. 0. 5. 6.]
 [1. 1. 3. 0. 1.]]
[[1. 2. 0. 0. 0.]
 [0. 4. 0. 5. 6.]
 [1. 1. 3. 0. 1.]]


### 6.1.2 - Question 5 (1.5 points)

*Implement the bag-of-words model that weights the vector with the absolute word frequency.*

For this exercise, except scipy.sparse.csc_matrix, you cannot use any external python library (e.g., scikit-learn).

In [11]:
from scipy.sparse import csr_matrix
from collections import Counter

def transform_count_bow(X):
    """
    This method preprocesses the data using the pipeline object, relates each token to a specific integer and  
    transforms the text in a vector. Vectors are weighted using the token frequencies in the sentence.

    X: document tokens. e.g: [['I','will', 'be', 'back', '.'], ['Helllo', 'world', '!'], ['If', 'you', 'insist', 'on', 'using', 'a', 'damp', 'cloth']]

    :return: vector representation of each document (document-term matrix)
    
    """
    data = []
    indices = []
    indptr = [0]
    voc = {}
    
    
    for doc in X:
        doc_counter = Counter(doc)
        for token in doc_counter.keys():
            if token not in voc:
                voc[token] = 0 if len(indices) == 0 else max(indices) + 1
            data.append(doc_counter[token])
            indices.append(voc[token])
        indptr.append(len(indices))
    return csr_matrix((data, indices, indptr), shape=(len(X), max(indices) + 1))

#test
X = [['I','will', 'be', 'back', '.'], ['Helllo', 'Helllo', '!'], ['Helllo', 'you', 'insist', 'on', 'using', 'a', 'damp', 'cloth']]
print(transform_count_bow(X).toarray())


[[1 1 1 1 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 2 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 1 1 1 1 1 1]]


## 6.2 - TF-IDF


Using raw frequency in the bag-of-words can be problematic. The word frequency distribution
is skewed - only a few words have high frequencies in a document. Consequently, the
weight of these words will be much bigger than the other ones which can give them more
impact on some tasks, like similarity comparison. Besides that, a set of words (including
those with high frequency) appears in most of the documents and, therefore, they do not
help to discriminate documents. For instance, the word *of* appears in a significant
part of tweets. Thus, having the word *of* does not make
documents more or less similar. However, the word *terrible* is rarer and documents that
have this word are more likely to be negative. TF-IDF is a technique that overcomes the word frequency disadvantages.


TF-IDF weights the vector using inverse document frequency (IDF) and word frequency, called term frequency (TF).
TF is the local information about how important is a word to a specific document.  IDF measures the discrimination level of the words in a dataset.  Common words in a domain are not helpful to discriminate documents since most of them contain these terms. So, to reduce their relevance in the documents, these words should have low weights in the vectors . 
The following equation calculates the word IDF:
\begin{equation}
	idf_i = \log\left( \frac{N}{df_i} \right),
\end{equation}
where $N$ is the number of documents in the dataset, $df_i$ is the number of documents that contain a word $i$.
The new weight $w_{ij}$ of a word $i$ in a document $j$ using TF-IDF is computed as:
\begin{equation}
	w_{ij} = tf_{ij} \times idf_i,
\end{equation}
where $tf_{ij}$ is the term frequency of word $i$ in the document $j$.



### 6.2.1 - Question 6 (2 points)

*Implement a bag-of-words model that weights the vector using TF-IDF.*

For this exercise, except scipy.sparse.csc_matrix, you cannot use any external python library (e.g., scikit-learn).

In [12]:
import math

def transform_tf_idf_bow(X):
    """
    This method preprocesses the data using the pipeline object, calculates the IDF and TF and 
    transforms the text in vectors. Vectors are weighted using TF-IDF method.

    X: document tokens. e.g: [['I','will', 'be', 'back', '.'], ['Helllo', 'world', '!'], ['If', 'you', 'insist', 'on', 'using', 'a', 'damp', 'cloth']]

    :return: vector representation of each document
    """
    idf = {}
    df = {}
    w = []
    indptr = [0]
    indices = []
    voc = {}
    
    for doc in X:
        for token in Counter(doc).keys():
            #Check the number of documents that countain the token (word) and store the value in df
            df[token] = df.get(token, 0) + 1
    
    #Calculating value idfi with formula
    for i, dfi in df.items(): 
        idf[i] = math.log(len(X)/dfi, 10)
    
    #Calculating value wij with formula
    for doc in X:
        doc_counter = Counter(doc)
        for token in doc_counter.keys():
            w.append((doc_counter[token]/len(doc)) * idf[token] )
            if token not in voc:
                voc[token] = 0 if len(indices) == 0 else max(indices) + 1
            indices.append(voc[token])
        indptr.append(len(indices))
    return csr_matrix((w, indices, indptr), shape=(len(X), max(indices) + 1))
    
#test
print(transform_tf_idf_bow(X).toarray())
        

[[0.09542425 0.09542425 0.09542425 0.09542425 0.09542425 0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.11739417
  0.15904042 0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.         0.         0.         0.02201141
  0.         0.05964016 0.05964016 0.05964016 0.05964016 0.05964016
  0.05964016 0.05964016]]


# 7 - Our System

## 7.1 - Pipeline

The pipeline is a sequence of preprocessing steps that transform the raw data to a format that is suitable for your problem. In this TP, our pipeline is composed of the following steps:

1. Concatenate the summary and description
2. Perform the tokenization, stop word removal and stemming in textual data
3. Generate the vector representation using transform_tf_idf_bow or transform_count_bow
4. Encode the categorical data (component and product) to integers


The pipeline (nlp_pipeline function) receives the a list of bug reports (dictionary that contains the report information), tokenization type, vectorizer type, a flag that enables or disable the insignificant token removal and a flag that turn stemming on or off. The function nlp_pipeline function returns a tuple ($p$, $c$, $M$):
- $p$ is a vector that contains the product values of the bug reports
- $c$ is a vector that contains the component values of the bug reports
- $M$ is matrix with the text representation.

The $i$-th element of $p$, $c$ and $M$ are the product, component and text representation of the $i$-th report in bug_reports, respectively.


In [13]:
from scipy.sparse import csr_matrix
from collections import defaultdict

def nlp_pipeline(bug_reports, tokenization_type, vectorizer_type, enable_stop_words, enable_stemming):
    """
    Preprocess and vectorize the threads.
    
    bug_reports: list of all bug reports([dict()]).
    tokenization_type: two possible values "space_tokenization" and "nltk_tokenization".
                            - space_tokenization: tokenize_space function is used to tokenize.
                            - nltk_tokenization: tokenize_nltk function is used to tokenize.
                            - space_punk_tokenization: tokenize_space_punk is used to tokenize.
                            
    vectorizer_type: two possible values "count" and "tf_idf".
                            - count: use transform_count_bow to vectorize the text
                            - tf_idf: use transform_tf_idf_bow to vectorize the text
                            
    enable_stop_words: Enables or disables the insignificant stop words removal
    enable_stemming: Enables or disables steming
    
    return: tuple ($p$, $c$, $M$)
    """
    all_thread_ids = []
    X = []
    
    if tokenization_type == 'space_tokenization':
        tkn_func = tokenize_space
    elif tokenization_type == 'nltk_tokenization':
        tkn_func = tokenize_nltk
    elif tokenization_type == 'space_punk_tokenization':
        tkn_func = tokenize_space_punk
    
    product_vocab = defaultdict(int)
    component_vocab = defaultdict(int)
    
    p = np.zeros((len(bug_reports),))
    c = np.zeros((len(bug_reports),))

    for idx, report in enumerate(bug_reports):
        product_vocab.setdefault(report['product'], len(product_vocab))
        component_vocab.setdefault(report['component'], len(component_vocab))
        
        p[idx] = product_vocab[report['product']]
        c[idx] = component_vocab[report['component']]
                 
        text = report['summary'] +"\n" + report["description"]
        tkns =  tkn_func(text)
        
        if enable_stop_words:
            tkns = filter_tokens(tkns)
            
        if enable_stemming:
            tkns = (stemmer.stem(tkn) for tkn in tkns)
                    
        X.append(list(tkns))
        
    
    if vectorizer_type == 'count':
        vectorizer_func = transform_count_bow
    elif vectorizer_type == 'tf_idf':
        vectorizer_func = transform_tf_idf_bow
        
    X = vectorizer_func(X)
    
    return p, c, X
    
    
    

## 7.2 - Question 7 (1 point)

*Implement the function rank that returns a list of reports indexes sorted by similarity of the bug reports (candidates) and new bug report (query)*. We use the following similarity function to compare two bug reports:

$$
\mathrm{SIM}(q,r) = w_1 * F_1(q,r) + w_2 * F_c(q,r) + w_3 * cos\_sim(\mathrm{txt}_q, \mathrm{txt}_c),
$$
$$
 F_p(q,r) = \begin{cases}
    1 ,& \text{if } p_q= p_r\\
    0,              & \text{otherwise},
\end{cases}
$$
$$
 F_p(q,r) = \begin{cases}
    1 ,& \text{if } c_q = c_r\\
    0,              & \text{otherwise},
\end{cases}
$$
where $c_q$ and $c_r$ are the query and candidate components,
 $p_q$ and $p_r$ are the query and candidate products,
 $\mathrm{txt}_q$ and $\mathrm{txt}_c$ are the query and candidate textual representation, and
 w_1, w_2 and w_3 are parameters to be tuned.
 

**In this question, the query have to  be removed in the sorted list (rank output).**

*For better performance, you can use the cosine similarity from a library (e.g, [sklearn](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html)). Also, we recommend to perform matrix operations.*


In [14]:
from sklearn.metrics.pairwise import cosine_similarity

def rank(query_idx, p, c, M, w1, w2, w3):
    """
    Return a list of reports indexes sorted by similarity of the bug reports (candidates) and new bug report (query)
    Cosine similarity is used to compare bug reports. 
    
    query_idx: query indexes
    p: product values of all bug reports (list)
    c: component values of all bug reports  (list)
    M: textual data representation of all bug reports  (Matrix)
    
    w1: parameter that controls the impact of the product
    w2: parameter  that controls the impact of the component
    w3: parameter  that controls the impact of textual similrity
    
    return: ranked list of indexes. 
    """
    Fp = [1 if p[query_idx] == pr else 0 for pr in p]
    Fc = [1 if c[query_idx] == cr else 0 for cr in c]
    cos_sim = cosine_similarity(M, M[query_idx])
    sim = list(zip(range(len(p)), Fp, Fc, cos_sim))
    sim.pop(query_idx)
    res = [int(x[0]) for x in sorted(sim, key=lambda x:((w1*x[1])+(w2*x[2])+(w3*(x[3][0]))), reverse=True)]
     
    return res



## 7.3 - Evaluation

We will test different configurations of our recommender system. These configurations are compared using the [mean average precision (MAP) metric](https://en.wikipedia.org/wiki/Evaluation_measures_(information_retrieval)#Mean_average_precision). Basically, the closer relevant threads are from ranked list begining, the higher MAP is. Additional materials to undertand MAP: [recall and precision over ranks](https://youtu.be/H7oAofuZjjE) and [MAP](https://youtu.be/pM6DJ0ZZee0).


The function *eval* evaluates a specific configurantion of our system



In [15]:
from statistics import mean 


def calculate_map(x):
    res = 0.0
    n = 0.0
    
    for query_id, corrects, candidate_ids in x:
        precisions = []
        for k, candidate_id in enumerate(candidate_ids):
            
            if candidate_id in corrects:
                prec_at_k = (len(precisions) + 1)/(k+1)
                precisions.append(prec_at_k)
                
            if len(precisions) == len(corrects):
                break
                    
        res += mean(precisions)
        n += 1
    
    return res/n
            

def eval(tokenization_type, vectorizer, enable_stop_words, enable_stemming, w1=0.1, w2=0.1, w3=2):
    reports = [r for r in report_index.values()]
    report_ids = [r["report_id"] for r in report_index.values()]
    prod_v, comp_v, M = nlp_pipeline(reports, tokenization_type, vectorizer, enable_stop_words, enable_stemming)
    report2idx = dict([(r['report_id'], idx) for idx,r in enumerate(reports)])
    rank_lists = []

    for query_id, corrects in test:
        query_idx =  report_ids.index(query_id)
        candidate_idxs = rank(query_idx, prod_v, comp_v, M, w1, w2, w3)
        candidate_ids = [ report_ids[idx] for idx in candidate_idxs]
        rank_lists.append((query_id, corrects, candidate_ids))
        
    return calculate_map(rank_lists)
    

## 7.4 - Question 8  (2 points)


Evaluate the system precision(MAP) using each one of the following configurations:

1. count(BoW) + space_tokenization
2. count(BoW) + nltk_tokenization
3. count(BoW) + space_punk_tokenization
4. count(BoW) + space_punk_tokenization + Stop words removal
5. count(BoW) + space_punk_tokenization + Stop words removal + Stemming
6. tf_idf + space_punk_tokenization
7. tf_idf + space_punk_tokenization + Stop words removal
8. tf_idf + space_punk_tokenization + Stop words removal + Stemming 

In all configurations, you have to use $w_1=0.1$, $w_2=0.1$ and $w_3=2.0$. 

Each performance below the following MAP values will be penalized by 0.2 points

1. count(BoW) + space_tokenization: 0.086 
2. count(BoW) + nltk_tokenization: 0.092
3. count(BoW) + space_punk_tokenization: 0.130
4. count(BoW) + space_punk_tokenization + Stop words removal: 0.170
5. count(BoW) + space_punk_tokenization + Stop words removal + Stemming: 0.195
6. tf_idf + space_punk_tokenization: 0.210
7. tf_idf + space_punk_tokenization + Stop words removal: 0.210
8. tf_idf + space_punk_tokenization + Stop words removal + Stemming: 0.217

In [16]:
#1.
print(eval('space_tokenization', 'count', False, False))

0.0921376847643323


In [17]:
#2.
print(eval('nltk_tokenization', 'count', False, False))

0.09605791029808612


In [18]:
#3.
print(eval('space_punk_tokenization', 'count', False, False))

0.132064688688776


In [19]:
#4.
print(eval('space_punk_tokenization', 'count', True, False))

0.17688406098757295


In [20]:
#5.
print(eval('space_punk_tokenization', 'count', True, True))

0.19674909366309945


In [21]:
#6.
print(eval('space_punk_tokenization', 'tf_idf', False, False))

0.21049970135353585


In [22]:
#7.
print(eval('space_punk_tokenization', 'tf_idf', True, False))

0.20850645367394177


In [23]:
#8.
print(eval('space_punk_tokenization', 'tf_idf', True, True))

0.22107804598292818


**Also, describe and compare the results found by you and answer the following questions:**

After running the tests above, we obtained these results:

|Test # | tokenization_type | vectorizer | enable_stop_words | enable_stemming | MAP |
|:-:|:-:|:-:|:-:|:-:|:-:|
|1 | space_tokenization  | count  | False  | False | 0.0921376847643323 | 
|2 | nltk_tokenization  | count  | False  | False | 0.09605791029808612 | 
|3 | space_punk_tokenization  | count  | False  | False | 0.132064688688776 | 
|4 | space_punk_tokenization  | count  | True  | False | 0.17688406098757295 | 
|5 | space_punk_tokenization  | count  | True  | True | 0.19674909366309945 |
|6 | space_punk_tokenization  | tf_idf  | False  | False | 0.21049970135353585 | 
|7 | space_punk_tokenization  | tf_idf  | True  | False | 0.20850645367394177 | 
|8 | space_punk_tokenization  | tf_idf  | True  | True | 0.22107804598292818 | 

When we briefly go over the table, we can see that space_punk_tokenization generates the highest results, and hence does a more accurate comparison. On the other hand, space_tokenization has the lowest MAP result and hence the least accurate tokenization method. As for the nltk_tokenization, it is a little bit more effective than the space_tokenization but a lot less accurate than space_punk_tokenization. 

**- Why do you think space_punk_tokenization achieve better results than the two other tokenization strategies?**

As a reminder, here are the test results from the question 5.2.1:

space_tokenization: ['hello', 'world', 'of', 'nlp']
["it's", 'the', "student's", 'notebook.']

nltk_tokenization: ['hello', 'world', 'of', 'nlp']
['it', "'s", 'the', 'student', "'s", 'notebook', '.']


space_punk_tokenization: ['hello', 'world', 'of', 'nlp']
['it', 's', 'the', 'student', 's', 'notebook']

From these results, we can affirm that the space_punk_tokenization strategy uses more constraints to generate the tokens. These constraints include the punctuation marks and spaces from the English language. Therefore, it is more effective to find similarities between sets of tokens since there won't be any punctuation marks lingering after a word (ex.: "notebook." from space_tokenization). Moreover, the vocabulary won't consider punctuations marks in the vocabulary, as nltk_tokenization does. This will increase our system's efficiency since punctuations marks are pretty common in English, and hence not relevant tokens.

**- Why is the stop word removal not clearly effective in the TF-IDF configurations?**

As a reminder, here is how the TF-IDF technique work:
\begin{equation}
    idf_i = \log\left( \frac{N}{df_i} \right),
\end{equation}
where $N$ is the number of documents in the dataset, $df_i$ is the number of documents that contain a word $i$.
The new weight $w_{ij}$ of a word $i$ in a document $j$ using TF-IDF is computed as:
\begin{equation}
    w_{ij} = tf_{ij} \times idf_i,
\end{equation}

Based on the equations above, the more a token has occured in the documents, the more it will have a low idf and a low weight. For example, if all the documents contain the same token, this token will have an idf and a weight of zero (0).

As for the stop word removal, the most common words in the English language are being removed.

We can see that the TF-IDF technique has a similar behaviour as the stop word removal. On one hand, the weight of the most frequent words in the documents is reduced, and on the other hand these words are being ignored. Therefore, the stop word removal has little to no effect when using the TF-IDF technique.